## Setup

<a href="https://colab.research.google.com/github/Devasy23/FaceRec/blob/feature-triplet-loss-function/Model-Training/Facerec_final_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deepface wandb tensorflow-addons keras huggingface_hub

# Downloading dataset

In [2]:
!wget http://vis-www.cs.umass.edu/lfw/lfw.tgz

--2024-07-19 08:52:41--  http://vis-www.cs.umass.edu/lfw/lfw.tgz
Resolving vis-www.cs.umass.edu (vis-www.cs.umass.edu)... 128.119.244.95
Connecting to vis-www.cs.umass.edu (vis-www.cs.umass.edu)|128.119.244.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180566744 (172M) [application/x-gzip]
Saving to: ‘lfw.tgz’

lfw.tgz             100%[===================>] 172.20M   682KB/s    in 6m 2s   

2024-07-19 08:58:45 (487 KB/s) - ‘lfw.tgz’ saved [180566744/180566744]



In [ ]:
!tar -xzvf lfw.tgz

write code to clean the dataset where if any folder in lfw/ has less than 4 images then delete the directory

In [4]:
import os
import shutil
count=0
for folder in os.listdir('lfw'):
    if len(os.listdir(f'lfw/{folder}')) < 4:
        shutil.rmtree(f'lfw/{folder}')
        count+=1
print(f"Removed {count} folders")

Removed 5139 folders


In [5]:
print(len(os.listdir('lfw')))

610


Import Modules

In [6]:
import deepface
from deepface.basemodels.Facenet import load_facenet512d_model
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

In [7]:
def get_embedding_module(imageSize=160):

    # Load Facenet Model
    model = load_facenet512d_model()
    model.trainable = False
    inputs = keras.Input(shape=(imageSize, imageSize, 3))

    # Resizing the input
    x = layers.Resizing(160, 160)(inputs)
    output = model(x)
    embedding = keras.Model(inputs, output, name="embedding")
    return embedding

In [8]:
print(deepface.__version__)

0.0.92


In [9]:
import os

directory = '/root/.deepface/weights'
os.makedirs(directory, exist_ok=True)

In [10]:
def get_siamese_network(imageSize, embeddingModel):
    # build the anchor, positive and negative input layer
    anchorInput = keras.Input(name="anchor", shape=imageSize + (3,))
    positiveInput = keras.Input(name="positive", shape=imageSize + (3,))
    negativeInput = keras.Input(name="negative", shape=imageSize + (3,))
    # embed the anchor, positive and negative images
    anchorEmbedding = embeddingModel(anchorInput)
    positiveEmbedding = embeddingModel(positiveInput)
    negativeEmbedding = embeddingModel(negativeInput)
    # build the siamese network and return it
    siamese_network = keras.Model(
        inputs=[anchorInput, positiveInput, negativeInput],
        outputs=[anchorEmbedding, positiveEmbedding, negativeEmbedding]
    )
    return siamese_network

In [11]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Mean
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

In [12]:
class TripletDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_paths, labels, batch_size, image_size, num_classes):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.image_size = image_size
        self.num_classes = num_classes
        self.label_encoder = LabelEncoder()
        self.encoded_labels = self.label_encoder.fit_transform(labels)
        self.image_data_generator = ImageDataGenerator()
        self.on_epoch_end()
        print(f"Initialized TripletDataGenerator with {len(self.image_paths)} images")

    def __len__(self):
        return max(1, len(self.image_paths) // self.batch_size)  # Ensure at least one batch

    def __getitem__(self, index):
        batch_image_paths = self.image_paths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labels = self.encoded_labels[index * self.batch_size:(index + 1) * self.batch_size]
        return self._generate_triplet_batch(batch_image_paths, batch_labels)

    def on_epoch_end(self):
        # Shuffle the data at the end of each epoch
        combined = list(zip(self.image_paths, self.encoded_labels))
        np.random.shuffle(combined)
        self.image_paths[:], self.encoded_labels[:] = zip(*combined)

    def _generate_triplet_batch(self, batch_image_paths, batch_labels):
        anchor_images = []
        positive_images = []
        negative_images = []

        for i in range(len(batch_image_paths)):
            anchor_path = batch_image_paths[i]
            anchor_label = batch_labels[i]

            positive_path = np.random.choice(
                [p for p, l in zip(self.image_paths, self.encoded_labels) if l == anchor_label]
            )
            negative_path = np.random.choice(
                [p for p, l in zip(self.image_paths, self.encoded_labels) if l != anchor_label]
            )

            anchor_image = load_img(anchor_path, target_size=self.image_size)
            positive_image = load_img(positive_path, target_size=self.image_size)
            negative_image = load_img(negative_path, target_size=self.image_size)

            anchor_images.append(img_to_array(anchor_image))
            positive_images.append(img_to_array(positive_image))
            negative_images.append(img_to_array(negative_image))

        return (
            {
                "anchor": np.array(anchor_images),
                "positive": np.array(positive_images),
                "negative": np.array(negative_images)
            },
            None,
        )

In [13]:
class SiameseModel(keras.Model):
    def __init__(self, siameseNetwork, margin, lossTracker):
        super().__init__()
        self.siameseNetwork = siameseNetwork
        self.margin = margin
        self.lossTracker = lossTracker

    def _compute_distance(self, inputs):
        anchor, positive, negative = inputs["anchor"], inputs["positive"], inputs["negative"]
        # embed the images using the siamese network
        embeddings = self.siameseNetwork((anchor, positive, negative))
        anchorEmbedding = embeddings[0]
        positiveEmbedding = embeddings[1]
        negativeEmbedding = embeddings[2]
        # calculate the anchor to positive and negative distance
        apDistance = tf.reduce_sum(
            tf.square(anchorEmbedding - positiveEmbedding), axis=-1
        )
        anDistance = tf.reduce_sum(
            tf.square(anchorEmbedding - negativeEmbedding), axis=-1
        )
        # return the distances
        return (apDistance, anDistance)

    def _compute_loss(self, apDistance, anDistance):
        loss = apDistance - anDistance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss

    def call(self, inputs):
        # compute the distance between the anchor and positive,
        # negative images
        apDistance, anDistance = self._compute_distance(inputs)
        return (apDistance, anDistance)

    def train_step(self, inputs):
        with tf.GradientTape() as tape:
            # compute the distance between the anchor and positive,
            # negative images
            apDistance, anDistance = self._compute_distance(inputs)
            # calculate the loss of the siamese network
            loss = self._compute_loss(apDistance, anDistance)
        # compute the gradients and optimize the model
        gradients = tape.gradient(
            loss,
            self.siameseNetwork.trainable_variables)
        self.optimizer.apply_gradients(
            zip(gradients, self.siameseNetwork.trainable_variables)
        )
        # update the metrics and return the loss
        self.lossTracker.update_state(loss)
        return {"loss": self.lossTracker.result()}

    def test_step(self, inputs):
        # compute the distance between the anchor and positive,
        # negative images
        apDistance, anDistance = self._compute_distance(inputs)
        # calculate the loss of the siamese network
        loss = self._compute_loss(apDistance, anDistance)
        # update the metrics and return the loss
        self.lossTracker.update_state(loss)
        return {"loss": self.lossTracker.result()}

    @property
    def metrics(self):
        return [self.lossTracker]

    def get_config(self):
        config = super().get_config()
        config.update({
            "siameseNetwork": self.siameseNetwork,
            "margin": self.margin,
            "lossTracker": self.lossTracker
            })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

# Now running the model

In [31]:
# Set the directory structure
data_dir = 'lfw'
image_size = (160, 160)
batch_size = 32  # Adjust the batch size for the small dataset
margin = 0.6

In [32]:
# Initialize W&B
wandb.init(project="FaceRec", config={
    "learning_rate": 0.00001,
    "epochs": 15,
    "batch_size": 32,
    "optimizer": "Adam",
    "architecture": "FaceNet",
    "dataset": "lfw",
    "loss": "TripletLoss",
    "margin": 0.6
})

batch/batch_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁
batch/batch_step,2545
batch/learning_rate,1e-05
batch/loss,0.49995
epoch/epoch,14


In [33]:
# Load and preprocess the data
image_paths = []
labels = []

for label in os.listdir(data_dir):
    label_dir = os.path.join(data_dir, label)
    if os.path.isdir(label_dir):
        for image_name in os.listdir(label_dir):
            image_paths.append(os.path.join(label_dir, image_name))
            labels.append(label)

In [34]:
# Debugging output
print(f"Found {len(image_paths)} images in {len(set(labels))} classes")

# Split the data into training and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

# Check if the splits are non-empty
print(f"Training on {len(train_paths)} images")
print(f"Validating on {len(val_paths)} images")

Found 6733 images in 610 classes
Training on 5386 images
Validating on 1347 images


In [35]:
# Create data generators
num_classes = len(set(labels))
train_generator = TripletDataGenerator(train_paths, train_labels, batch_size, image_size, num_classes)
val_generator = TripletDataGenerator(val_paths, val_labels, batch_size, image_size, num_classes)

# Check if the generators have data
assert len(train_generator) > 0, "Training generator is empty!"
assert len(val_generator) > 0, "Validation generator is empty!"

Initialized TripletDataGenerator with 5386 images
Initialized TripletDataGenerator with 1347 images


In [36]:
def make_model_finetunable(embeddingModel, layersToTrain):
    embeddingModel.trainable = True
    for layer in embeddingModel.layers[:-layersToTrain]:
        layer.trainable = False
    return embeddingModel

In [37]:
# Create the embedding model and the Siamese network
embedding_model = get_embedding_module(image_size[0])
embedding_model = make_model_finetunable(embedding_model, 10)
siamese_network = get_siamese_network(image_size, embedding_model)

# Initialize the Siamese model
loss_tracker = Mean(name="loss")
siamese_model = SiameseModel(siamese_network, margin, loss_tracker)

# Compile the model
siamese_model.compile(optimizer=Adam(1e-5))

In [38]:
# Train the model
siamese_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    callbacks=[WandbMetricsLogger(log_freq=5), WandbModelCheckpoint("models")]
)

Epoch 1/15
168/168 [==============================] - ETA: 0s - loss: 105.2571

wandb: Adding directory to artifact (./models)... Done. 1.5s


168/168 [==============================] - 217s 965ms/step - loss: 105.2571 - val_loss: 9.0454
Epoch 2/15
168/168 [==============================] - ETA: 0s - loss: 7.5691

wandb: Adding directory to artifact (./models)... Done. 1.4s


168/168 [==============================] - 154s 917ms/step - loss: 7.5691 - val_loss: 2.4464
Epoch 3/15
168/168 [==============================] - ETA: 0s - loss: 2.0882

...

168/168 [==============================] - 152s 906ms/step - loss: 0.3479 - val_loss: 0.2854
Epoch 14/15
168/168 [==============================] - ETA: 0s - loss: 0.3425

wandb: Adding directory to artifact (./models)... Done. 2.0s


168/168 [==============================] - 160s 951ms/step - loss: 0.3425 - val_loss: 0.2119
Epoch 15/15
168/168 [==============================] - ETA: 0s - loss: 0.3371

wandb: Adding directory to artifact (./models)... Done. 1.4s


168/168 [==============================] - 153s 908ms/step - loss: 0.3371 - val_loss: 0.2628


## Saving the model

In [39]:
embedding_model.save("embedding_trial3.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [29]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [41]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="/content/embedding_trial3.h5",
    path_in_repo="embedding_trial3.h5",
    repo_id="DnD11/FaceNet_Finetuned",
    repo_type="model",
)

embedding_trial3.h5:   0%|          | 0.00/94.7M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DnD11/FaceNet_Finetuned/commit/93074cb19f105958bb79f786a640e14cf42d2b69', commit_message='Upload embedding_trial3.h5 with huggingface_hub', commit_description='', oid='93074cb19f105958bb79f786a640e14cf42d2b69', pr_url=None, pr_revision=None, pr_num=None)